Primero leemos los ficheros iniciales de entrada.

In [24]:
import pandas as pd
import numpy
from sklearn.preprocessing import StandardScaler
from sklearn import model_selection

In [47]:
pcosData = pd.read_csv('./datos/pcos.csv', skiprows = 1, header = None,
                           names=['Age (yrs)', 'Weight (Kg)', 'Height(Cm)', 'BMI', 'Blood Group', 'Pulse rate(bpm)',
                                   'RR (breaths/min)', 'Hb(g/dl)', 'Cycle(R/I)', 'Cycle length(days)', 'Marriage Status (Yrs)',
                                     'Pregnant(Y/N)', 'No. of abortions', 'I beta-HCG(mIU/mL)', 'FSH(mIU/mL)', 'LH(mIU/mL)', 'FSH/LH', 'Hip(inch)',
                                       'Waist(inch)', 'Waist:Hip Ratio', 'TSH (mIU/L)', 'PRL(ng/mL)', 'Vit D3 (ng/mL)', 'PRG(ng/mL)', 'RBS(mg/dl)', 'Weight gain(Y/N)', 'hair growth(Y/N)',
                                         'Skin darkening (Y/N)', 'Hair loss(Y/N)', 'Pimples(Y/N)', 'Fast food (Y/N)', 'Reg.Exercise(Y/N)', 'BP _Systolic (mmHg)', 'BP _Diastolic (mmHg)', 'Follicle No. (L)',
                                           'Follicle No. (R)', 'Avg. F size (L) (mm)', 'Avg. F size (R) (mm)', 'Endometrium (mm)', 'PCOS (Y/N)'])


titanicData = pd.read_csv('./datos/titanic.csv', skiprows = 1, header = None,
                              names=['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked', 'Initial', 'Age_band', 'Family_Size', 'Alone', 'Fare_cat', 'Deck', 'Title', 'Is_Married', 'Survived'])


Definimos la funcion para estandarizar los ficheros.

In [42]:

def dataStandarization (data, dataName, excludedColumns):
    if not isinstance (dataName, str):
        print("El argumento dataName debe ser una cadena de carateres.")
    
    elif not isinstance (excludedColumns, list):
        print("El argumento excludedColumns debe ser una lista con el nombre de las columnas a excluir.")

    else:
        
        data_copy = data.drop(excludedColumns, axis = 1)
        numerical_columns = [col for col in data.columns if col not in excludedColumns]
        columns_to_standarize = data[numerical_columns]

        scaler = StandardScaler()
        columns_to_standarize = scaler.fit_transform(columns_to_standarize)
        data[numerical_columns] = columns_to_standarize

        data.to_csv('./datos/' + dataName + 'Standarized.csv', index = False)

        print("El nuevo fichero estandarizado se ha guardado en el directorio datos con el nombre " + dataName + ".csv")


Definimos las columnas que no vamos a considerar para la estandarización, en este caso las columnas booleanas y las columnas resultado.

In [43]:
titanic_excluded = ['Sex', 'Alone', 'Is_Married', 'Survived']
pcos_excluded = ['Pregnant(Y/N)', 'Weight gain(Y/N)', 'hair growth(Y/N)', 'Skin darkening (Y/N)', 'Hair loss(Y/N)', 'Pimples(Y/N)', 'Fast food (Y/N)','Reg.Exercise(Y/N)', 'PCOS (Y/N)']

dataStandarization(titanicData, 'titanic', titanic_excluded)
dataStandarization(pcosData, 'pcos', pcos_excluded)

El nuevo fichero estandarizado se ha guardado en el directorio datos con el nombre titanic.csv
El nuevo fichero estandarizado se ha guardado en el directorio datos con el nombre pcos.csv


A continuación dividiremos los ficheros estandarizados en dos, uno para el conjunto de entrenamiento y otro para el conjunto de prueba. Para ello definiremos la función necesaria.

In [50]:
def dataSplit (data, dataName):
    if not isinstance (dataName, str):
        print("El argumento dataName debe ser una cadena de carateres.")
    
    else:
        data_train, data_test = model_selection.train_test_split(data, test_size = 0.3, random_state = 99)
        data_train.to_csv('./datos/' + dataName + '_train.csv', index = False)
        data_test.to_csv('./datos/' + dataName + '_test.csv', index = False)

        print('Las dimensiones originales de los datos de entrada son: ', data.shape)
        print('El conjunto de entrenamiento se ha guardado en el directorio datos con el nombre ' + dataName + '_train.csv, y sus dimensiones son: ', data_train.shape)
        print('El conjunto de pruebas se ha guardado en el directorio datos con el nombre ' + dataName + '_test.csv, y sus dimensiones son: ', data_test.shape)





Finalmente creamos los dos conjuntos de datos a partir del fichero estandarizado:

In [54]:
dataSplit(titanicData, 'titanic')
dataSplit(pcosData, 'pcos')

Las dimensiones originales de los datos de entrada son:  (891, 16)
El conjunto de entrenamiento se ha guardado en el directorio datos con el nombre titanic_train.csv, y sus dimensiones son:  (623, 16)
El conjunto de pruebas se ha guardado en el directorio datos con el nombre titanic_test.csv, y sus dimensiones son:  (268, 16)
Las dimensiones originales de los datos de entrada son:  (541, 40)
El conjunto de entrenamiento se ha guardado en el directorio datos con el nombre pcos_train.csv, y sus dimensiones son:  (378, 40)
El conjunto de pruebas se ha guardado en el directorio datos con el nombre pcos_test.csv, y sus dimensiones son:  (163, 40)
